In [10]:
from pathlib import Path
import pandas as pd

# Rutas robustas (igual que antes)
PROJECT_ROOT = Path.cwd()
if PROJECT_ROOT.name == "notebooks":
    PROJECT_ROOT = PROJECT_ROOT.parent

DATA_RAW = PROJECT_ROOT / "data" / "raw"

products = pd.read_csv(DATA_RAW / "products.csv")
tickets = pd.read_csv(DATA_RAW / "tickets.csv", parse_dates=["datetime"])
lines = pd.read_csv(DATA_RAW / "ticket_lines.csv", parse_dates=["datetime"])

print(products.shape, tickets.shape, lines.shape)
products.head()

(60, 8) (12568, 12) (52081, 8)


,product_id,product_name,category,currency,unit_price_gross,unit_price_net,food_cost_unit,prep_minutes
0,1,Bebidas Item 1,Bebidas,CLP,4508,3788,676,15
1,2,Fondos Item 2,Fondos,CLP,12711,10682,3963,16
2,3,Bebidas Item 3,Bebidas,CLP,4044,3398,436,5
3,4,Postres Item 4,Postres,CLP,6942,5834,1599,16
4,5,Entradas Item 5,Entradas,CLP,7356,6182,1579,3


In [11]:
df = (
    lines
    .merge(products, on="product_id", how="left")
    .merge(
        tickets[["ticket_id", "party_size", "ticket_total_gross", "ticket_total_net", "food_cost_total", "tip_amount", "channel"]],
        on=["ticket_id", "channel"],
        how="left"
    )
)

df["margin_net"] = df["line_total_net"] - df["line_cost"]
df["margin_pct_net"] = df["margin_net"] / df["line_total_net"]

df[["ticket_id","datetime","product_id","qty","line_total_net","line_cost","margin_net","channel"]].head()

,ticket_id,datetime,product_id,qty,line_total_net,line_cost,margin_net,channel
0,1,2025-11-01 20:41:00,6,1,2656,397,2259,dine_in
1,1,2025-11-01 20:41:00,12,1,2805,596,2209,dine_in
2,1,2025-11-01 20:41:00,49,1,3306,864,2442,dine_in
3,1,2025-11-01 20:41:00,2,1,10682,3963,6719,dine_in
4,1,2025-11-01 20:41:00,43,1,3946,986,2960,dine_in


In [12]:
kpis = {
    "tickets": tickets["ticket_id"].nunique(),
    "items_vendidos": int(lines["qty"].sum()),
    "ventas_brutas_CLP": int(tickets["ticket_total_gross"].sum()),
    "ventas_netas_CLP": int(tickets["ticket_total_net"].sum()),
    "food_cost_CLP": int(tickets["food_cost_total"].sum()),
}

kpis["margen_neto_CLP"] = kpis["ventas_netas_CLP"] - kpis["food_cost_CLP"]
kpis["food_cost_pct"] = kpis["food_cost_CLP"] / kpis["ventas_netas_CLP"]
kpis["ticket_prom_neto_CLP"] = kpis["ventas_netas_CLP"] / kpis["tickets"]

kpis

{'tickets': 12568,
 'items_vendidos': 66746,
 'ventas_brutas_CLP': 456537528,
 'ventas_netas_CLP': 383644390,
 'food_cost_CLP': 109043168,
 'margen_neto_CLP': 274601222,
 'food_cost_pct': 0.28422979937227805,
 'ticket_prom_neto_CLP': 30525.49252068746}

In [2]:
from pathlib import Path

# 1) Encontrar raíz del proyecto (sube 1 nivel desde /notebooks)
PROJECT_ROOT = Path.cwd()
if PROJECT_ROOT.name == "notebooks":
    PROJECT_ROOT = PROJECT_ROOT.parent

DATA_RAW = PROJECT_ROOT / "data" / "raw"
DATA_PROCESSED = PROJECT_ROOT / "data" / "processed"

DATA_RAW.mkdir(parents=True, exist_ok=True)
DATA_PROCESSED.mkdir(parents=True, exist_ok=True)

print("PROJECT_ROOT:", PROJECT_ROOT)
print("DATA_RAW:", DATA_RAW)
print("DATA_PROCESSED:", DATA_PROCESSED)

PROJECT_ROOT: c:\Users\danni\Documents\DataScient\proyecto_restaurant\restaurant-demand-optimization
DATA_RAW: c:\Users\danni\Documents\DataScient\proyecto_restaurant\restaurant-demand-optimization\data\raw
DATA_PROCESSED: c:\Users\danni\Documents\DataScient\proyecto_restaurant\restaurant-demand-optimization\data\processed


In [1]:
# Config (adaptable)
CURRENCY = "CLP"
VAT_RATE = 0.19           # Chile base, pero configurable
TIP_RATE = 0.10           # opcional
INCLUDE_TIP_IN_REVENUE = False

CHANNELS = ["dine_in", "takeaway", "delivery"]
N_SKUS_RANGE = (40, 80)   # típico configurable

In [3]:
# Guarda CSVs en data/raw
products_df.to_csv(DATA_RAW / "products.csv", index=False)
tickets_df.to_csv(DATA_RAW / "tickets.csv", index=False)
lines_df.to_csv(DATA_RAW / "ticket_lines.csv", index=False)

print("Saved files:")
for p in (DATA_RAW / "products.csv", DATA_RAW / "tickets.csv", DATA_RAW / "ticket_lines.csv"):
    print(" -", p, "| exists:", p.exists(), "| size:", p.stat().st_size if p.exists() else None)

NameError: name 'products_df' is not defined

In [4]:
import pandas as pd

dfs = []
for name, obj in globals().items():
    if isinstance(obj, pd.DataFrame):
        dfs.append((name, obj.shape))

dfs

RuntimeError: dictionary changed size during iteration

In [5]:
import pandas as pd

dfs = []
for name, obj in globals().items():
    if isinstance(obj, pd.DataFrame):
        dfs.append((name, obj.shape, list(obj.columns)))

print(dfs)

[]


In [6]:
import numpy as np
import pandas as pd

# Config adaptable
CURRENCY = "CLP"
VAT_RATE = 0.19
TIP_RATE = 0.10
INCLUDE_TIP_IN_REVENUE = False

CHANNELS = ["dine_in", "takeaway", "delivery"]
N_PRODUCTS = 60  # típico entre 40 y 80 (puedes cambiarlo)
SEED = 42

rng = np.random.default_rng(SEED)

In [7]:
def generate_products(n_products: int, rng: np.random.Generator) -> pd.DataFrame:
    categories = ["Entradas", "Fondos", "Postres", "Bebidas", "Café"]
    cat_probs = np.array([0.18, 0.42, 0.12, 0.22, 0.06])

    product_id = np.arange(1, n_products + 1)
    category = rng.choice(categories, size=n_products, p=cat_probs)

    base_price = {
        "Entradas": (4500, 9000),
        "Fondos": (8000, 18000),
        "Postres": (3500, 8000),
        "Bebidas": (1500, 6000),
        "Café": (1500, 4500),
    }
    unit_price_gross = np.array([rng.uniform(*base_price[c]) for c in category]).round(0).astype(int)
    unit_price_net = (unit_price_gross / (1 + VAT_RATE)).round(0).astype(int)

    # Food cost unit (CLP): % por categoría
    fc_range = {
        "Entradas": (0.25, 0.38),
        "Fondos": (0.28, 0.42),
        "Postres": (0.22, 0.35),
        "Bebidas": (0.12, 0.22),
        "Café": (0.10, 0.20),
    }
    fc_pct = np.array([rng.uniform(*fc_range[c]) for c in category])
    food_cost_unit = (unit_price_net * fc_pct).round(0).astype(int)  # costo sobre neto

    prep_minutes = rng.integers(2, 18, size=n_products)
    product_name = [f"{category[i]} Item {product_id[i]}" for i in range(n_products)]

    return pd.DataFrame({
        "product_id": product_id,
        "product_name": product_name,
        "category": category,
        "currency": CURRENCY,
        "unit_price_gross": unit_price_gross,
        "unit_price_net": unit_price_net,
        "food_cost_unit": food_cost_unit,
        "prep_minutes": prep_minutes
    })

products_df = generate_products(N_PRODUCTS, rng)
products_df.head()

,product_id,product_name,category,currency,unit_price_gross,unit_price_net,food_cost_unit,prep_minutes
0,1,Bebidas Item 1,Bebidas,CLP,4508,3788,676,15
1,2,Fondos Item 2,Fondos,CLP,12711,10682,3963,16
2,3,Bebidas Item 3,Bebidas,CLP,4044,3398,436,5
3,4,Postres Item 4,Postres,CLP,6942,5834,1599,16
4,5,Entradas Item 5,Entradas,CLP,7356,6182,1579,3


In [8]:
def generate_tickets_and_lines(
    products_df: pd.DataFrame,
    start_date: str = "2025-11-01",
    days: int = 120,
    rng: np.random.Generator | None = None
) -> tuple[pd.DataFrame, pd.DataFrame]:

    if rng is None:
        rng = np.random.default_rng(42)

    product_ids = products_df["product_id"].to_numpy()
    price_g = products_df.set_index("product_id")["unit_price_gross"].to_dict()
    price_n = products_df.set_index("product_id")["unit_price_net"].to_dict()
    cost_u = products_df.set_index("product_id")["food_cost_unit"].to_dict()

    open_hours = np.arange(11, 24)  # 11..23
    hour_weights = np.array([0.03, 0.04, 0.08, 0.10, 0.08, 0.05, 0.04, 0.05, 0.08, 0.12, 0.15, 0.12, 0.06])
    hour_weights = hour_weights / hour_weights.sum()

    dates = pd.date_range(start=start_date, periods=days, freq="D")
    ticket_rows, line_rows = [], []
    ticket_id = 1

    for d in dates:
        # Cerrado lunes (configurable)
        if d.day_name() == "Monday":
            continue

        dow = d.dayofweek  # 0 lunes ... 6 domingo
        dow_multiplier = {1: 1.00, 2: 1.02, 3: 1.05, 4: 1.15, 5: 1.30, 6: 1.20}.get(dow, 1.0)

        base_tickets = int(rng.integers(80, 140))
        n_tickets = int(base_tickets * dow_multiplier)

        for _ in range(n_tickets):
            hour = int(rng.choice(open_hours, p=hour_weights))
            minute = int(rng.integers(0, 60))
            dt = pd.Timestamp(year=d.year, month=d.month, day=d.day, hour=hour, minute=minute)

            channel = rng.choice(CHANNELS, p=[0.78, 0.12, 0.10])
            party_size = int(rng.choice([1, 2, 3, 4, 5, 6], p=[0.05, 0.22, 0.28, 0.25, 0.12, 0.08]))

            mean_items = 1.2 * party_size
            n_lines = max(1, int(rng.poisson(lam=mean_items)))

            # Popularidad Zipf (pocos productos venden mucho)
            popularity = np.arange(1, len(product_ids) + 1)
            weights = (1 / popularity)
            weights = weights / weights.sum()

            chosen = rng.choice(product_ids, size=n_lines, replace=True, p=weights)

            ticket_total_g = 0
            ticket_total_n = 0
            ticket_cost = 0

            for pid in chosen:
                pid = int(pid)
                qty = int(rng.choice([1, 1, 1, 2, 2, 3], p=[0.45, 0.20, 0.10, 0.15, 0.07, 0.03]))
                line_total_g = int(price_g[pid] * qty)
                line_total_n = int(price_n[pid] * qty)
                line_cost = int(cost_u[pid] * qty)

                ticket_total_g += line_total_g
                ticket_total_n += line_total_n
                ticket_cost += line_cost

                line_rows.append({
                    "ticket_id": ticket_id,
                    "datetime": dt,
                    "product_id": pid,
                    "qty": qty,
                    "line_total_gross": line_total_g,
                    "line_total_net": line_total_n,
                    "line_cost": line_cost,
                    "channel": channel
                })

            tip_amount = int(round(ticket_total_g * TIP_RATE)) if (channel == "dine_in") else 0
            total_with_tip = ticket_total_g + tip_amount

            ticket_rows.append({
                "ticket_id": ticket_id,
                "datetime": dt,
                "party_size": party_size,
                "channel": channel,
                "currency": CURRENCY,
                "vat_rate": VAT_RATE,
                "ticket_total_gross": ticket_total_g,
                "ticket_total_net": ticket_total_n,
                "food_cost_total": ticket_cost,
                "tip_rate": TIP_RATE,
                "tip_amount": tip_amount,
                "total_with_tip": total_with_tip
            })

            ticket_id += 1

    tickets_df = pd.DataFrame(ticket_rows)
    lines_df = pd.DataFrame(line_rows)
    return tickets_df, lines_df

tickets_df, lines_df = generate_tickets_and_lines(products_df, start_date="2025-11-01", days=120, rng=rng)

tickets_df.head(), lines_df.head()

(   ticket_id            datetime  party_size   channel currency  vat_rate  \
 0          1 2025-11-01 20:41:00           5   dine_in      CLP      0.19   
 1          2 2025-11-01 20:10:00           3  takeaway      CLP      0.19   
 2          3 2025-11-01 21:55:00           3   dine_in      CLP      0.19   
 3          4 2025-11-01 13:11:00           2   dine_in      CLP      0.19   
 4          5 2025-11-01 14:49:00           4   dine_in      CLP      0.19   
 
    ticket_total_gross  ticket_total_net  food_cost_total  tip_rate  \
 0               36856             30971             8158       0.1   
 1                9016              7576             1352       0.1   
 2               61040             51295            16377       0.1   
 3                9016              7576             1352       0.1   
 4               51624             43382            12201       0.1   
 
    tip_amount  total_with_tip  
 0        3686           40542  
 1           0            9016  
 2 

In [9]:
products_df.to_csv(DATA_RAW / "products.csv", index=False)
tickets_df.to_csv(DATA_RAW / "tickets.csv", index=False)
lines_df.to_csv(DATA_RAW / "ticket_lines.csv", index=False)

print("Saved to:", DATA_RAW)
for f in ["products.csv", "tickets.csv", "ticket_lines.csv"]:
    p = DATA_RAW / f
    print(f" - {f} | exists={p.exists()} | size={p.stat().st_size if p.exists() else None}")

Saved to: c:\Users\danni\Documents\DataScient\proyecto_restaurant\restaurant-demand-optimization\data\raw
 - products.csv | exists=True | size=3022
 - tickets.csv | exists=True | size=961961
 - ticket_lines.csv | exists=True | size=2807043


In [13]:
copy notebooks\01_generate_dataset.ipynb notebooks\02_eda_kpis_basico.ipynb

El sistema no puede encontrar la ruta especificada.
